# The Basics of Applied Numerical Methods Using Python

## HOMEWORK 3

**EXAMPLE 2.17**

Write a computer program to solve the following $n$ simultaneous equations by the Gauss-Seidel method with relaxation (the program should work with any value of $n$):

$$
\left[\begin{array}{rrrrcrrrr}
 2 & -1 &  0 &  0 & \dots &  0 &  0 &  0 &  1\\
-1 &  2 & -1 &  0 & \dots &  0 &  0 &  0 &  0\\
 0 & -1 &  2 & -1 & \dots &  0 &  0 &  0 &  0\\
 \vdots & \vdots & \vdots & \vdots && \vdots & \vdots & \vdots & \vdots \\
 0 &  0 &  0 &  0 & \dots & -1 &  2 & -1 &  0\\
 0 &  0 &  0 &  0 & \dots &  0 & -1 &  2 & -1\\
 1 &  0 &  0 &  0 & \dots &  0 &  0 & -1 &  2\\
\end{array}\right]
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ \vdots \\ x_{n-2} \\ x_{n-1} \\ x_n
\end{bmatrix}=\begin{bmatrix}
0\\0\\0\\\vdots\\0\\0\\1
\end{bmatrix}
$$

Run the program with $n=20$. The exact solution can be shown to be $x_i=-n/4+i/2$,$i=1,2,\dots,n$.

**Solution** In this case the iterative formulas derived as

$$
\left\{\begin{aligned}
2x_1-x_2+x_n &=0\\
-x_{i-1}+2x_i-x_{i+1} &=0\\
x_1-x_{n-1}+2x_n &=1
\end{aligned}\right\} .
$$

$$
\Rightarrow\left\{
\begin{aligned}
x_1&=\frac{x_2-x_n}{2}\\
x_i&=\frac{x_{i-1}+x_{i+1}}{2}\\
x_n&=\frac{1-x_1+x_{n-1}}{2}
\end{aligned}\right . \Rightarrow\left\{\begin{aligned}
x_1 &= \frac{\omega\left(x_2-x_n\right)}{2}+\left(1-\omega\right)x_1\\
x_i &= \frac{\omega\left(x_{i-1}+x_{i+1}\right)}{2}+\left(1-\omega\right)x_i\\
x_n &= \frac{\omega\left(1-x_1+x_{n-1}\right)}{2}+\left(1-\omega\right)x_n
\end{aligned}\right\} .
$$

$$i=2,3,\dots,n-1$$

which are evaluated by the following function:

In [ ]:
def fex2_17(x_, omega_):
    # Iteration formula Eq. (2.35) for Example 2.17 on Numerical Methods in
    # Engineering with MATLAB. 2nd Edition

    n_ = len(x_)
    x_[0] = omega_ * (x_[1] - x_[n_ - 1]) / 2.0 + (1.0 - omega_) * x_[0]
    for i_ in range(1, n_ - 1):
        x_[i_] = omega_ * (x_[i_ - 1] + x_[i_ + 1]) / 2.0 + (1.0 - omega_) * x_[i_]

    x_[n_ - 1] = omega_ * (1 - x_[0] + x_[n_ - 2]) / 2.0 + (1.0 - omega_) * x_[n_ - 1]

    return x_

In [ ]:
import numpy as np

def gaussSeidel_relax(func, x_, *args):
    """Solves Ax = b by Gauss-Seidel method with relaxation.
    USAGE: [x, numIter, omega] = gaussSeidel(func, x, maxIter, epsilon)
    INPUT:
    func    = handle of function that returns improved x using
              the iterative formulas
    x       = starting solution vector
    maxIter = allowable number of iterations (default is 500)
    epsilon = error tolerance (default is 1.0e-9)
    OUTPUT:
    x       = solution vector
    numIter = number of iterations carried out
    omega   = computed relaxation factor"""

    if len(args) < 2:
        epsilon = 1.0e-9
    if len(args) < 1:
        maxIter = 5000
    k = 10
    p = 1
    omega_ = 1
    for numIter_ in range(maxIter):
        xOld = x_.copy()
        x_ = func(x_, omega_)
        dx = np.sqrt(np.dot(x_ - xOld, x_ - xOld))
        if dx < epsilon:
            return x_, numIter_, omega_
        if numIter_ == k:
            dx1 = dx
        if numIter_ == k + p:
            omega_ = 2.0 / (1.0 + np.sqrt(1.0 - (dx / dx1) ** (1.0 / p)))

    print('Too many iterations')
    return x_, maxIter, omega_

The solution can be obtained with a single commmand (note that the $\mathbf{x}=\mathbf{0}$ is the starting vector):

In [ ]:
[x, numIter, omega] = gaussSeidel_relax(fex2_17, np.zeros(20))
print('result using Gauss-Seidel:\n', x)
print('\nnumIter = ', numIter)
print('\nomega =', omega)

Speed comparison between Gauss Seidel method and `numpy.linalg.solve` to solve Example 2.17:

In [ ]:
import time
import matplotlib.pyplot as plt

N = 20
m = np.round(np.logspace(1, 2, N)).astype(np.int)
time_cost = np.zeros((N, 2))

ntimes = 5
for i in range(len(m)):
    A = np.eye(m[i])
    b = np.zeros(m[i])
    b[-1] = 1
    A = A * 2
    for n in range(1, m[i]):
        A[n, n - 1] = -1   # A = A + diag(eye(n-1)*-1, -1);
        A[n-1, n] = -1     # A = A + diag(eye(n-1)*-1, 1);

    A[0, -1] = 1
    A[-1, 0] = 1

    start = time.time()
    for n in range(ntimes):
        np.linalg.solve(A, b)

    time_cost[i, 0] = (time.time() - start) / ntimes

    start = time.time()
    for n in range(ntimes):
        gaussSeidel_relax(fex2_17, np.zeros(m[i]))

    time_cost[i, 1] = (time.time() - start) / ntimes

plt.loglog(m, time_cost[:, 0], 'bo-', m, time_cost[:, 1], 'rx-')
plt.title('Speed comparison of matrix ...')
plt.legend(('linalg.solve', 'Gauss Seidel'))
plt.show()

**PROBLEM SET 2.3**

In [ ]:
# 1.
import numpy as np

A = np.array([[3,-1,2], [0,1,3], [-2,2,-4]])
print('inv(A) =\n', np.linalg.inv(A))
B = np.array([[0,1,3], [3,-1,2], [-2,2,-4]])
print('\n inv(B) =\n', np.linalg.inv(B))

In [ ]:
# 2.
A = np.array([[2,4,3], [0,6,5], [0,0,2]])
B = np.array([[2,0,0], [3,4,0], [4,5,6]])
print('inv(A) =\n', np.linalg.inv(A))
print('\ninv(B) =\n', np.linalg.inv(B))

In [ ]:
# 3.
A = np.array([[1,1/2,1/4,1/8], [0,1,1/3,1/9], [0,0,1,1/4], [0,0,0,1]])
print('inv(A) =\n', np.linalg.inv(A))

In [ ]:
# 4.
A = np.array([[1,2,4], [1,3,9], [1,4,16]])
B = np.array([[4,-1,0], [-1,4,-1], [0,-1,4]])
print('inv(A) =\n', np.linalg.inv(A))
print('\ninv(B) =\n', np.linalg.inv(B))

In [ ]:
# 5.
A = np.array([[4,-2,1], [-2,1,-1], [1,-2,4]])
print('inv(A) =\n', np.linalg.inv(A))

In [ ]:
# 6.
A = np.array([[5,-3,-1,0], [-2,1,1,1], [3,-5,1,2], [0,8,-4,-3]])
B = np.array([[4,-1,0,0], [-1,4,-1,0], [0,-1,4,-1], [0,0,-1,4]])
print('inv(A) =\n', np.linalg.inv(A))
print('\ninv(B) =\n', np.linalg.inv(B))

In [ ]:
# 7.
A = np.array([[1,3,-9,6,4], [2,-1,6,7,1], [3,2,-3,15,5], [8,-1,1,4,2], [11,1,-2,18,7]])
print('inv(A) =\n', np.linalg.inv(A))

In [ ]:
# 9.
A = np.array([[3,-7,45,21], [12,11,10,17], [6,25,-80,-24], [17,55,-9,7]])
B = np.array([[1,1,1,1], [1,2,2,2], [2,3,4,4], [4,5,6,7]])
print('inv(A) =\n', np.linalg.inv(A))
print('\ninv(B) =\n', np.linalg.inv(B))

In [ ]:
def inv_LTM(A):
    """inverting a lower triangular matrix
    PROBLEM SET 2.3 on Numerical Methods in Engineering with MATLAB, 2nd
    Edition, P95, 10
    input: A = the lower triangular matrix to invert
    output: x = the inversion of A"""

    m, n = np.size(A, 0), np.size(A, 1)
    if m != n:
        print('The input should be square matrix!')
        return None

    x = np.eye(n)
    x[0, 0] = x[0, 0] / A[0, 0]
    for i in range(1, n):
        for j in range(i+1):
            x[i, j] = (x[i, j] - A[i, 0 : i].dot(x[0 : i, j])) / A[i, i]
    
    return x

In [ ]:
# 10.
A = np.array([[36,0,0,0], [18,36,0,0], [9,12,36,0], [5,4,9,36]])
print('inv(A) =\n', np.linalg.inv(A))
print('\ninv_LTM(A) =\n', inv_LTM(A))

In [ ]:
# 11.
def gaussSeidel(func, x_, *args):
    """Solves Ax = b by Gauss-Seidel method
    USAGE: [x, numIter] = gaussSeidel(func, x, maxIter, epsilon)
    INPUT:
    func    = handle of function that returns improved x using
              the iterative formulas
    x       = starting solution vector
    maxIter = allowable number of iterations (default is 500)
    epsilon = error tolerance (default is 1.0e-9)
    OUTPUT:
    x       = solution vector
    numIter = number of iterations carried out"""

    if len(args) < 2:
        epsilon = 1.0e-9
    if len(args) < 1:
        maxIter = 5000
    for numIter_ in range(maxIter):
        xOld = x_.copy()
        x_ = func(x_)
        dx = np.sqrt(np.dot(x_ - xOld, x_ - xOld))
        if dx < epsilon:
            return x_, numIter_

    print('Too many iterations')
    return x_, maxIter


A = np.array([[-2, 5, 9], [7, 1, 1], [-3, 7, -1]])
b = np.array([1, 6, -26])

def fp11(x):
    """Iteration formula for Problem 11 on Numerical Methods in
    Engineering with MATLAB. 2nd Edition, P95"""

    n = len(x)

    for i in range(n):
        ind = list(range(0,i)) + list(range(i+1,n))
        x[i] = (b[i] - A[i, ind].dot(x[ind])) / A[i, i]
    
    return x

x = gaussSeidel(fp11, np.zeros(3))
print('Since coefficient matrix A is not diagonally dominant, Gauss-Seidel mthod does not converge.')
print(np.linalg.solve(A, b))

In [ ]:
# 12.
A = np.array([[12,-2,3,1], [-2,15,6,-3], [1,6,20,-4], [0,-3,2,9]])
b = np.array([0, 0, 20, 0])
 
def fp12(x):
    """Iteration formula for Problem 12 on Numerical Methods in
    Engineering with MATLAB. 2nd Edition, P95"""

    n = len(x)
       
    for i in range(n):
        ind = list(range(0,i)) + list(range(i+1,n))
        x[i] = (b[i] - A[i, ind].dot(x[ind])) / A[i, i]
    
    return x

x, numIter = gaussSeidel(fp12, np.zeros(4))
print('numIter =', numIter)
print('\nresult using Gauss-Seidel:\n', x)
print('\nresult using numpy.linalg.solve:\n', np.linalg.solve(A, b))

In [ ]:
# 13.
A = np.array([[4, -1, 0, 0], [-1, 4, -1, 0], [0, -1, 4, -1], [0, 0, -1, 3]])
b = np.array([15, 10, 10, 10])
def fp13(x, omega):
    """Iteration formula for Problem 13 on Numerical Methods in
    Engineering with MATLAB. 2nd Edition, P95"""

    n = len(x)

    for i in range(n):
        ind = list(range(0,i)) + list(range(i+1,n))
        x[i] = omega * (b[i] - A[i, ind].dot(x[ind])) / A[i, i] + (1 - omega) * x[i]
    
    return x

x, numIter, omega = gaussSeidel_relax(fp13, np.zeros(4))
print('numIter =', numIter)
print('omega =', omega)
print('\nresult using Gauss-Seidel:\n', x)
print('\nresult using numpy.linalg.solve:\n', np.linalg.solve(A, b))

In [ ]:
# 16.
A = np.array([[3,-2,1,0,0,0],[-2,4,-2,1,0,0],[1,-2,4,-2,1,0],[0,1,-2,4,-2,1],[0,0,1,-2,4,-2],[0,0,0,1,-2,3]])
B = np.array([[3,-2,1,0,0,1],[-2,4,-2,1,0,0],[1,-2,4,-2,1,0],[0,1,-2,4,-2,1],[0,0,1,-2,4,-2],[1,0,0,1,-2,3]])
b = np.array([10, -8, 10, 10, -8, 10])

def fp16(x, omega):
    """Iteration formula for Problem 16 on Numerical Methods in
    Engineering with MATLAB. 2nd Edition, P96"""

    n = len(x)

    for i in range(n):
        ind = list(range(0,i)) + list(range(i+1,n))
        x[i] = omega * (b[i] - A[i, ind].dot(x[ind])) / A[i, i] + (1 - omega) * x[i]
    
    return x

x, numIter, omega = gaussSeidel_relax(fp16, np.zeros(6))
print('numIter =', numIter)
print('omega =', omega)
print('\nresult using Gauss-Seidel:\n', x)
print('\nresult using numpy.linalg.solve:\n', np.linalg.solve(A, b))

In [ ]:
# 17.
def fp17(x, omega):
    """Iteration formula for Problem 17 on Numerical Methods in
    Engineering with MATLAB. 2nd Edition, P95"""

    n = len(x)
    
    x[0] = omega * (x[1] - x[n-1]) / 4 + (1 - omega) * x[0]
    for i in range(1, n-1):
        x[i] = omega * (x[i-1] + x[i+1]) / 4 + (1 - omega) * x[i]

    x[n-1] = omega * (100 - x[0] + x[n-2]) / 4 + (1 - omega) * x[n-1]
    
    return x

x, numIter, omega = gaussSeidel_relax(fp17, np.zeros(20))
print('numIter =', numIter)
print('omega =', omega)
print('\nresult using Gauss-Seidel:\n', x)

n = 20
A = np.eye(n)
b = np.zeros(n)
b[-1] = 100
A = A * 4
for n in range(1, n):
    A[n, n - 1] = -1   # A = A + diag(eye(n-1)*-1, -1);
    A[n-1, n] = -1     # A = A + diag(eye(n-1)*-1, 1);

A[0, -1] = 1
A[-1, 0] = 1

print('\nresult using numpy.linalg.solve:\n', np.linalg.solve(A, b))

In [ ]:
# 19.
# Iteration formula for Problem 19 on Numerical Methods in
# Engineering with MATLAB. 2nd Edition, P97

M = np.array([[-4, 1, 0], [1, -4, 1], [0, 1, -4]])
I = np.eye(3)
Z = np.zeros((3,3))
A = np.hstack((M, I, Z))
A = np.vstack((A, np.hstack((I, M, I))))
A = np.vstack((A, np.hstack((Z, I, M))))

b = np.array([0, 0, 100, 0, 0, 100, 200, 200, 300])

T = np.linalg.solve(A, -b)
print('T =\n', T)
print(A)